<a href="https://colab.research.google.com/github/rraulison/olx/blob/main/scrap_olx_cars_2023_by_GPT4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install bs4

In [10]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36",
}

def get_pages(url):
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.content, "html.parser")

  if response.status_code != 200:
          print(f"HTTP status code {response.status_code} for URL {url}.")
          return None

  script_tag = soup.find('script', {'id': '__NEXT_DATA__'})

  if not script_tag:
      print("Error: Cannot find script tag in HTML.")
      return None

  try:
    data = json.loads(script_tag.get_text(strip=True))
  except json.JSONDecodeError as e:
    print("Error parsing JSON:", e)
    return None

  return data

urls = [f'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/estado-ce?cf=5&me=120000&pe=40000&ps=20000&rs=61&o={i}'
        for i in range(1,21)]

def process_data(df):

  categories = ['vehicle_model', 'vehicle_brand', 'regdate', 'mileage',
                'motorpower', 'gearbox', 'car_steering', 'car_features',
                'cartype']

  entries = {category: None for category in categories}

  data = {category: [] for category in categories}

  data['price'] = []

  for ad in df.get('props', {}).get('pageProps', {}).get('ads', []):

    for prop in ad.get('properties', []):

      if prop['name'] in categories:
        value = prop['value']

        if prop['name'] in ['regdate', 'mileage']:
          try:
            value = int(value)
          except ValueError:
            continue

        entries[prop['name']] = value

    ad_price = ad.get('price', '').replace("R$", "").replace(".","")

    try:
      ad_price = int(ad_price)
    except ValueError:
      continue

    for category in categories:
      data[category].append(entries[category])

    data['price'].append(ad_price)

    entries = {category: None for category in categories}

  return data

dfs = []
for url in urls:
  page_data = get_pages(url)

  if page_data is not None:
      df = pd.DataFrame(process_data(page_data))
      dfs.append(df)

resultado = pd.concat(dfs, ignore_index=True)

In [9]:
resultado

,vehicle_model,vehicle_brand,regdate,mileage,motorpower,gearbox,car_steering,car_features,cartype,price
0,FORD KA 1.0 SE 12V FLEX 4P MANUAL,FORD,2017,82000,1.0,Manual,Elétrica,"Alarme, Air bag, Sensor de ré, Som, Trava elét...",Hatch,36900
1,CHEVROLET SPIN LT 1.8 8V ECONO.FLEX 5P MEC.,CHEVROLET,2014,180,1.8,Manual,Hidráulica,"Alarme, Air bag, Ar condicionado, Som, Trava e...",SUV,38500
2,FIAT UNO VIVACE CELEB. 1.0 EVO F.FLEX 8V 5P,FIAT,2015,120000,1.0,Manual,Hidráulica,"Alarme, Air bag, Ar condicionado, Som, Trava e...",Hatch,29900
3,FORD FOCUS TITA/TITA PLUS 2.0 FLEX 5P AUT.,FORD,2015,105000,2.0 - 2.9,Automático,Hidráulica,"Alarme, Air bag, Ar condicionado, Blindado, Câ...",Hatch,39990
4,FIAT PALIO ATTRACTIVE 1.0 EVO FIRE FLEX 8V 5P,FIAT,2016,50000,1.0,Manual,Hidráulica,"Alarme, Air bag, Ar condicionado, Som, Trava e...",Hatch,39990
...,...,...,...,...,...,...,...,...,...,...
1043,CHEVROLET COBALT LT 1.4 8V FLEXPOWER/ECONOFLEX 4P,CHEVROLET,2013,118000,1.4,Manual,Hidráulica,"Alarme, Air bag, Ar condicionado, Câmera de ré...",Sedã,30000
1044,RENAULT KWID 1.0 ZEN,RENAULT,2018,108000,1.0,Manual,Elétrica,"Alarme, Air bag, Ar condicionado, Sensor de ré...",Hatch,37500
1045,CHEVROLET HATCH LTZ 1.0 12V TB FLEX 5P AU,CHEVROLET,2020,102987,1.0,Manual,Hidráulica,"Alarme, Air bag, Ar condicionado, Blindado, Câ...",Hatch,34900
1046,RENAULT SANDERO AUTHENTIQUE HI-FLEX 1.0 16V 5P,RENAULT,2014,114000,1.0,Manual,Hidráulica,"Alarme, Air bag, Ar condicionado, Som, Trava e...",Hatch,29990
